In [1]:
from fh_id_data import fh_id_data
from fh_ca import campo_amostral
from fh_filter import get_filter
from fh_live_data import fh_live_data
from fh_id_predict import over_under
from send_telegram import enviar_mensagem_telegram, formatar_resultado
import os
from supabase import create_client, Client
import math
import logging
from apscheduler.schedulers.blocking import BlockingScheduler
table = 'table_statics'
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


In [ ]:

def filter_min(over_under_resp, min_percent=70, min_odd=1.2):

    dict_gols_over = over_under_resp[1]['gols'][0]
    dict_gols_under = over_under_resp[1]['gols'][1]
    dict_corners_over = over_under_resp[2]['corners'][0]
    dict_corners_under = over_under_resp[2]['corners'][1]
    dict_yellow_cards_over = over_under_resp[3]['yellowcards'][0]
    dict_yellow_cards_under = over_under_resp[3]['yellowcards'][1]
    dict_info = over_under_resp[0]
    list_gols_over_resp = []
    list_gols_under_resp = []
    list_corners_over_resp = []
    list_corners_under_resp = []
    list_yellow_cards_over_resp = []
    list_yellow_cards_under_resp = []
    dict_resposta = {}
    for key, value in dict_gols_over.items():
        if (value[2] >= min_percent) and (value[3] >= min_odd):
            list_gols_over_resp.append(dict_gols_over[key])


    for key, value in dict_gols_under.items():
        if (value[2] >= min_percent) and (value[3] >= min_odd):
            list_gols_under_resp.append(dict_gols_under[key])


    if len(list_gols_over_resp) > 0:
        dict_resposta['gols_over'] = list_gols_over_resp
    if len(list_gols_under_resp) > 0:
        dict_resposta['gols_under'] = list_gols_under_resp

    if len(list_corners_over_resp) > 0:
        dict_resposta['corners_over'] = list_corners_over_resp
    if len(list_corners_under_resp) > 0:
        dict_resposta['corners_under'] = list_corners_under_resp
    
    if len(list_yellow_cards_over_resp) > 0:
        dict_resposta['yellowcards_over'] = list_yellow_cards_over_resp
    if len(list_yellow_cards_under_resp) > 0:
        dict_resposta['yellowcards_under'] = list_yellow_cards_under_resp
    

    if len(list_gols_over_resp) + len(list_gols_under_resp) + len(list_corners_over_resp) + len(list_corners_under_resp) + len(list_yellow_cards_over_resp) + len(list_yellow_cards_under_resp)> 0:
        dict_resposta['info'] = dict_info
        


    return dict_resposta




def resposts_live(list_ids_live):
    list_resposts = []
    for id_live in list_ids_live:
        print(f'Processing live id: {id_live}')
        over_under_resp = over_under(id_live) 
        list_resposts.append(filter_min(over_under_resp))

    return list_resposts

def main_task():
    # list_ids_live = fh_live_data()
    list_ids_live = [14131963, 13472611,13472608]
    if len(list_ids_live) > 0:
        list_resposts_live = resposts_live(list_ids_live)
        mensagem = formatar_resultado(list_resposts_live)
        enviar_mensagem_telegram(mensagem)
    else:
        print('No live matches found.')



In [3]:
# list_ids_live = fh_live_data()
list_ids_live = [14131963, 13472611,13472608]

In [4]:
list_resposts_live = resposts_live(list_ids_live)
list_resposts_live

Processing live id: 14131963
Processing live id: 13472611
Processing live id: 13472608


[{'gols_under': [[4, 441, 78.3303730017762, 1.2766439909297052]],
  'info': {'event_id': 14131963,
   'id_tournament': 'Liga Portugal Betclic',
   'home_name': 'FC Porto',
   'away_name': 'Estoril Praia',
   'home_score': 1,
   'awayScore': 0,
   'home_statistics': {'Ball possession': 46,
    'Expected goals': 0.23,
    'Total shots': 2,
    'Shots on target': 2,
    'Shots inside box': 2,
    'Final third entries': 10,
    'cornerKicks': 1,
    'fouls': 11,
    'yellowCards': 1},
   'away_statistics': {'Ball possession': 54,
    'Expected goals': 0.99,
    'Total shots': 9,
    'Shots on target': 2,
    'Shots inside box': 5,
    'Final third entries': 28,
    'cornerKicks': 1,
    'fouls': 6,
    'yellowCards': 2}}},
 {'gols_under': [[4, 306, 74.63414634146342, 1.3398692810457515]],
  'info': {'event_id': 13472611,
   'id_tournament': 'Brasileirão Betano',
   'home_name': 'Corinthians',
   'away_name': 'Botafogo',
   'home_score': 1,
   'awayScore': 0,
   'home_statistics': {'Ball po

In [5]:
mensagem = formatar_resultado(list_resposts_live)
display(mensagem)

'<b>Partida: FC Porto vs Estoril Praia</b>\n\nTorneio: Liga Portugal Betclic\nPlacar intervalo: 1 - 0\nCorners intervalo: 1 - 1\nC. Amarelos intervalo: 1 - 2\n\nGols Under previsões:\n- Under 4: Prob 78.33%, Odd 1.28\n\n--------------------\n\n\n<b>Partida: Corinthians vs Botafogo</b>\n\nTorneio: Brasileirão Betano\nPlacar intervalo: 1 - 0\nCorners intervalo: 2 - 3\nC. Amarelos intervalo: 0 - 2\n\nGols Under previsões:\n- Under 4: Prob 74.63%, Odd 1.34\n\n--------------------\n\n\n<b>Partida: Fortaleza vs Atlético Mineiro</b>\n\nTorneio: Brasileirão Betano\nPlacar intervalo: 1 - 0\nCorners intervalo: 6 - 0\nC. Amarelos intervalo: 0 - 0\n\nGols Under previsões:\n- Under 4: Prob 75.22%, Odd 1.33\n\n--------------------\n'

In [6]:
enviar_mensagem_telegram(mensagem)

Mensagem enviada com sucesso via Telegram.


True